In [1]:
import pandas as pd
import requests

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

In [2]:
gkg = pd.read_csv("data/20240813.gkg.csv", sep="\t")
gkg.head()

,DATE,NUMARTS,COUNTS,THEMES,LOCATIONS,PERSONS,ORGANIZATIONS,TONE,CAMEOEVENTIDS,SOURCES,SOURCEURLS
0,20240813,1,NaN,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;TAX_FNCAC...,1#Canada#CA#CA#60#-96#CA;1#Israel#IS#IS#31.5#3...,charles krupa;laxman narasimhan;brian niccol;h...,pepsico;starbucks,"2.06185567010309,3.78006872852234,1.7182130584...",1192349779,castanetkamloops.net,https://www.castanetkamloops.net/news/Business...
1,20240813,1,NaN,MANMADE_DISASTER_IMPLIED;,"1#Brazil#BR#BR#-10#-55#BR;4#Sao Paulo, SãPaulo...",thalita valente machado;campos alencar e silva...,NaN,"-1.52542372881356,2.03389830508475,3.559322033...",NaN,bostonglobe.com,https://www.bostonglobe.com/2024/08/12/world/b...
2,20240813,1,NaN,SOC_GENERALCRIME;EPU_CATS_MIGRATION_FEAR_FEAR;...,"2#Indiana, United States#US#USIN#39.8647#-86.2...",shawn scudder;bob courtney,neighborhood watch program;madison police;madi...,"0.199600798403194,1.79640718562874,1.596806387...","1192464356,1192464356,1192501503,1192478508",wdrb.com,https://www.wdrb.com/news/crime-reports/city-o...
3,20240813,1,NaN,TAX_FNCACT;TAX_FNCACT_SPEAKER;UNGP_FORESTS_RIV...,"1#Pakistan#PK#PK#30#70#PK;4#Islamabad, Islamab...",andrei metelitsa,national assembly speaker sardar ayaz sadiq;pa...,"5.64102564102564,5.64102564102564,0,5.64102564...",1192348387,brecorder.com,https://www.brecorder.com/news/40317251/na-spe...
4,20240813,1,NaN,EPU_POLICY;EPU_POLICY_POLITICAL;USPEC_POLITICS...,"3#Washington, Washington, United States#US#USD...",michael l kripchak;michael rulli,penguin club;election commission;johnson leads...,"0.995024875621891,1.82421227197347,0.829187396...",NaN,tribtoday.com,https://www.tribtoday.com/news/local-news/2024...


In [3]:
# create a function to apply to counts to split the string AFFECT#3000000#Americans#1#United States#US#US#39.828175#-98.5795#US; into a list. the separator is ";"

def split_counts(counts, sep=";"):
    if pd.isna(counts):
        return []
    return counts[:-1].split(sep)

# apply the function to the counts column

gkg["counts"] = gkg["COUNTS"].apply(split_counts)
gkg["themes"] = gkg["THEMES"].apply(split_counts)
gkg["locations"] = gkg["LOCATIONS"].apply(split_counts)
gkg["persons"] = gkg["PERSONS"].apply(split_counts)
gkg["organizations"] = gkg["ORGANIZATIONS"].apply(split_counts)
gkg["tone"] = gkg["TONE"].apply(split_counts, sep=",")
gkg["cameoeventids"] = gkg["CAMEOEVENTIDS"].apply(split_counts, sep=",")
gkg["sources"] = gkg["SOURCES"].apply(split_counts)
gkg["sourcesurl"] = gkg["SOURCEURLS"].apply(split_counts, sep='<UDIV>')


In [4]:
def get_dicts_from_lists(counts):
    if len(counts) == 0:
        return []
    return [dict(zip(["COUNTTYPE", "NUMBER", "OBJECTTYPE", "GEO_TYPE", "GEO_FULLNAME", "GEO_COUNTRYCODE", "GEO_ADM1CODE", "GEO_LAT", "GEO_LONG", "GEO_FEATUREID"], i.split("#"))) for i in counts]

gkg["counts"] = gkg["counts"].apply(get_dicts_from_lists)

In [7]:
# expand gkg["sourcesurl"] to create a new row for each value in the list

gkg.explode("sourcesurl")["sourcesurl"].head()

0    https://www.castanetkamloops.net/news/Business...
1    https://www.bostonglobe.com/2024/08/12/world/b...
2    https://www.wdrb.com/news/crime-reports/city-o...
3    https://www.brecorder.com/news/40317251/na-spe...
4    https://www.tribtoday.com/news/local-news/2024...
Name: sourcesurl, dtype: object

In [12]:
header_url = "https://www.gdeltproject.org/data/lookups/CSV.header.dailyupdates.txt"
header = [h.strip() for h in requests.get(header_url).text.split("\t")]

In [15]:
events = pd.read_csv("data/20240813.export.CSV", sep="\t", header=None, names=header)
events.head()

/var/folders/tf/8v3l8mx557xf5pyvklc18qyr0000gn/T/ipykernel_26175/4102176926.py:1: DtypeWarning: Columns (14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv("data/20240813.export.CSV", sep="\t", header=None, names=header)


,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,Actor1Religion1Code,Actor1Religion2Code,Actor1Type1Code,Actor1Type2Code,Actor1Type3Code,Actor2Code,Actor2Name,Actor2CountryCode,Actor2KnownGroupCode,Actor2EthnicCode,Actor2Religion1Code,Actor2Religion2Code,Actor2Type1Code,Actor2Type2Code,Actor2Type3Code,IsRootEvent,EventCode,EventBaseCode,EventRootCode,QuadClass,GoldsteinScale,NumMentions,NumSources,NumArticles,AvgTone,Actor1Geo_Type,Actor1Geo_FullName,Actor1Geo_CountryCode,Actor1Geo_ADM1Code,Actor1Geo_Lat,Actor1Geo_Long,Actor1Geo_FeatureID,Actor2Geo_Type,Actor2Geo_FullName,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1192319188,20140816,201408,2014,2014.6192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GBR,UNITED KINGDOM,GBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,110,110,11,3,-2.0,10,2,10,3.289883,0,NaN,NaN,NaN,NaN,NaN,NaN,4,"Forest Of Bowland, Lancashire, United Kingdom",UK,UKH2,54.0000,-2.53333,-2590537,4,"Forest Of Bowland, Lancashire, United Kingdom",UK,UKH2,54.0000,-2.53333,-2590537,20240813,https://www.miragenews.com/312000-rural-homes-...
1,1192319189,20140816,201408,2014,2014.6192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GBR,UNITED KINGDOM,GBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,110,110,11,3,-2.0,560,110,560,1.733904,0,NaN,NaN,NaN,NaN,NaN,NaN,1,United Kingdom,UK,UK,54.0000,-4.00000,UK,1,United Kingdom,UK,UK,54.0000,-4.00000,UK,20240813,https://www.lancashiretelegraph.co.uk/news/nat...
2,1192319190,20230814,202308,2023,2023.6137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CVL,COMMUNITY,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,1,36,36,3,1,4.0,8,4,8,6.037992,0,NaN,NaN,NaN,NaN,NaN,NaN,3,"Lake Charles, Louisiana, United States",US,USLA,30.2266,-93.21740,1629966,3,"Lake Charles, Louisiana, United States",US,USLA,30.2266,-93.21740,1629966,20240813,https://107jamz.com/ixp/159/p/gordon-mckernan-...
3,1192319191,20230814,202308,2023,2023.6137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CVL,COMMUNITY,NaN,NaN,NaN,NaN,NaN,CVL,NaN,NaN,1,36,36,3,1,4.0,32,4,32,6.037992,0,NaN,NaN,NaN,NaN,NaN,NaN,2,"Louisiana, United States",US,USLA,31.1801,-91.87490,LA,2,"Louisiana, United States",US,USLA,31.1801,-91.87490,LA,20240813,https://107jamz.com/ixp/159/p/gordon-mckernan-...
4,1192319192,20230814,202308,2023,2023.6137,CHN,CHINA,CHN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MNG,MONGOLIA,MNG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,42,42,4,1,1.9,1,1,1,0.000000,1,China,CH,CH,35.0,105.0,CH,1,China,CH,CH,35.0000,105.00000,CH,1,China,CH,CH,35.0000,105.00000,CH,20240813,https://www.kmaj1440.com/news/fred-fleitz-to-n...


In [18]:
print(events["GLOBALEVENTID"].nunique())
print(events.shape)


132404
(132404, 58)


In [19]:
print(gkg.shape)

(103027, 11)


In [25]:
gkg[gkg["CAMEOEVENTIDS"]=='1192349779'].to_dict(orient="records")

[{'DATE': 20240813,
  'NUMARTS': 1,
  'COUNTS': nan,
  'THEMES': 'TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;TAX_FNCACT;TAX_FNCACT_CEO;TAX_FNCACT_CHAIRMAN;TAX_FNCACT_EXECUTIVE;GENERAL_HEALTH;MEDICAL;TAX_FNCACT_LEADER;RETIREMENT;WB_2690_CATEGORIES_OF_EMPLOYMENT;WB_2670_JOBS;WB_2689_JOBS_DIAGNOSTICS;WB_2896_RETIREMENT;ECON_BOYCOTT;TAX_FNCACT_DIRECTORS;SCIENCE;SOC_INNOVATION;UNGP_FORESTS_RIVERS_OCEANS;',
  'LOCATIONS': '1#Canada#CA#CA#60#-96#CA;1#Israel#IS#IS#31.5#34.75#IS;1#China#CH#CH#35#105#CH',
  'PERSONS': 'charles krupa;laxman narasimhan;brian niccol;howard schultz;mellody hobson',
  'ORGANIZATIONS': 'pepsico;starbucks',
  'TONE': '2.06185567010309,3.78006872852234,1.71821305841924,5.49828178694158,21.9931271477663,0.343642611683849',
  'CAMEOEVENTIDS': '1192349779',
  'SOURCES': 'castanetkamloops.net',
  'SOURCEURLS': 'https://www.castanetkamloops.net/news/Business/501194/Starbucks-replaces-its-CEO-names-Chipotle-chief-to-head-the-company'},
 {'DATE': 20240813,
  'NUMARTS': 1,
  'COUNTS'

In [23]:
events[events["GLOBALEVENTID"]==1192349779].to_dict(orient="records")

[{'GLOBALEVENTID': 1192349779,
  'SQLDATE': 20240813,
  'MonthYear': 202408,
  'Year': 2024,
  'FractionDate': 2024.611,
  'Actor1Code': 'CHN',
  'Actor1Name': 'CHINA',
  'Actor1CountryCode': 'CHN',
  'Actor1KnownGroupCode': nan,
  'Actor1EthnicCode': nan,
  'Actor1Religion1Code': nan,
  'Actor1Religion2Code': nan,
  'Actor1Type1Code': nan,
  'Actor1Type2Code': nan,
  'Actor1Type3Code': nan,
  'Actor2Code': nan,
  'Actor2Name': nan,
  'Actor2CountryCode': nan,
  'Actor2KnownGroupCode': nan,
  'Actor2EthnicCode': nan,
  'Actor2Religion1Code': nan,
  'Actor2Religion2Code': nan,
  'Actor2Type1Code': nan,
  'Actor2Type2Code': nan,
  'Actor2Type3Code': nan,
  'IsRootEvent': 1,
  'EventCode': 51,
  'EventBaseCode': 51,
  'EventRootCode': 5,
  'QuadClass': 1,
  'GoldsteinScale': 3.4,
  'NumMentions': 225,
  'NumSources': 173,
  'NumArticles': 225,
  'AvgTone': 1.1853114588699,
  'Actor1Geo_Type': 1,
  'Actor1Geo_FullName': 'China',
  'Actor1Geo_CountryCode': 'CH',
  'Actor1Geo_ADM1Code': 'CH'

In [20]:
gkg["GKGRECORDID"].nunique()

KeyError: 'GKGRECORDID'

In [6]:
from gdelt import GDELT

here


ImportError: cannot import name 'GDELT' from 'gdelt' (/Users/ahueteg/Documents/workspace/ahueteg/hackatons/factored-datathon-2024-ariteam/.venv/lib/python3.12/site-packages/gdelt/__init__.py)

In [8]:
from gdelt import gdelt as gdelt_client

In [10]:
client = gdelt_client(version=1)

In [11]:
graph = client.Search("2024 08 13", table="gkg")

In [13]:
graph.shape

(103027, 11)

In [12]:
graph.head()

,DATE,NUMARTS,COUNTS,THEMES,LOCATIONS,PERSONS,ORGANIZATIONS,TONE,CAMEOEVENTIDS,SOURCES,SOURCEURLS
1,20240813,1,NaN,TAX_ETHNICITY;TAX_ETHNICITY_CANADIAN;TAX_FNCAC...,1#Canada#CA#CA#60#-96#CA;1#Israel#IS#IS#31.5#3...,charles krupa;laxman narasimhan;brian niccol;h...,pepsico;starbucks,"2.06185567010309,3.78006872852234,1.7182130584...",1192349779,castanetkamloops.net,https://www.castanetkamloops.net/news/Business...
2,20240813,1,NaN,MANMADE_DISASTER_IMPLIED;,"1#Brazil#BR#BR#-10#-55#BR;4#Sao Paulo, SãPaulo...",thalita valente machado;campos alencar e silva...,NaN,"-1.52542372881356,2.03389830508475,3.559322033...",NaN,bostonglobe.com,https://www.bostonglobe.com/2024/08/12/world/b...
3,20240813,1,NaN,SOC_GENERALCRIME;EPU_CATS_MIGRATION_FEAR_FEAR;...,"2#Indiana, United States#US#USIN#39.8647#-86.2...",shawn scudder;bob courtney,neighborhood watch program;madison police;madi...,"0.199600798403194,1.79640718562874,1.596806387...","1192464356,1192464356,1192501503,1192478508",wdrb.com,https://www.wdrb.com/news/crime-reports/city-o...
4,20240813,1,NaN,TAX_FNCACT;TAX_FNCACT_SPEAKER;UNGP_FORESTS_RIV...,"1#Pakistan#PK#PK#30#70#PK;4#Islamabad, Islamab...",andrei metelitsa,national assembly speaker sardar ayaz sadiq;pa...,"5.64102564102564,5.64102564102564,0,5.64102564...",1192348387,brecorder.com,https://www.brecorder.com/news/40317251/na-spe...
5,20240813,1,NaN,EPU_POLICY;EPU_POLICY_POLITICAL;USPEC_POLITICS...,"3#Washington, Washington, United States#US#USD...",michael l kripchak;michael rulli,penguin club;election commission;johnson leads...,"0.995024875621891,1.82421227197347,0.829187396...",NaN,tribtoday.com,https://www.tribtoday.com/news/local-news/2024...


In [ ]:


# Initialize the GDELT object
gd = GDELT()

# Query the GDELT 2.0 events
events = gd.Search('2024-08-13', table='gkg', ve)  # You can specify dates, keywords, etc.